# Analyzing multiple states

`lnpy` is designed to analyze $\ln \Pi(N)$ at a single state point (temperature, volume, etc) across multiple values of $\ln z$.  To analyze across multiple state points, some extra work is required.  Here, we show an example of analyzing $\ln \Pi(N; T)$ for several values of the temperature $T$.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import lnpy


from tqdm.notebook import tqdm 
from joblib import Parallel, delayed

# load reference states

In [2]:
from pathlib import Path
import json 

def load_lnpi_from_json(path):
    with open(path, 'r') as f:
        ds = xr.Dataset.from_dict(json.load(f))
        
        if 'PE' in ds:
            extra_kws = {'PE': ds.PE.values}
        else:
            extra_kws = None
            
        ref = lnpy.lnPiMasked.from_dataarray(ds.lnpi, extra_kws=extra_kws)
    return ref
refs = [load_lnpi_from_json(p) for p in Path('./tmmc_json/').glob('*.json')]

# analyze a single lnpi

In [18]:
from lnpy.lnpicollectionutils import limited_collection

#function to tag 'LD' and 'HD' phases
def tag_phases2(x):
    if len(x) > 2:
        raise ValueError('bad tag function')
    else:
        argmax0 = np.array([xx.local_argmax()[0] for xx in x])
        if len(x) == 2:
            return np.argsort(argmax0)
        else:
            return np.where(argmax0 <= x[0].shape[0] / 2, 0, 1)



def get_stability(ref, nphase=2):
    
    
    phase_creator = lnpy.PhaseCreator(
        nmax=nphase, 
        nmax_peak=4, 
        ref=ref, 
        merge_kws={'efac': 0.5}, 
        tag_phases=tag_phases2
    )
    build_phases = phase_creator.build_phases_mu([None])

    # Turn off parallel here
    with lnpy.set_options(joblib_use=False, tqdm_use=False):    
        # If want nice mesh
#         c_course, c = limited_collection(
#             build_phases, 
#             dlnz=2.0, 
#             digits=3, 
#             offsets=[-10, +10], 
#             collection_kws=dict(unstack=False),
#             edge_distance_min=8,
#             dens_min=1e-6

#         )

        # For rough estimate, use something like:
        # NOTE: using a finer mesh like above might make calculation more accurate...
        lnzs = np.linspace(-10, +10, 10) + ref.lnz[0]
        c_course = lnpy.lnPiCollection.from_builder(lnzs=lnzs, build_phases=build_phases, unstack=False)
        
    
    
        # calculate spinodal
        efac = 1.0
        stability_kws = {
            'phase_ids': 2,
            'build_phases': build_phases,
            'build_kws': {'efac': efac * 0.5},
            'as_dict': False,
            'inplace': False,
        }

        try:
            spin, spin_info = c_course.spinodal(efac=efac, **stability_kws)
            bino, bino_info = c_course.binodal(spinodals=spin, **stability_kws)
        except:

            spin = bino = None

    return {
        'spin': spin, 
        'bino': bino,
    }


In [19]:
print('serial')
%timeit -n 1 -r 1 [get_stability(ref) for ref in refs]

serial
2.76 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
print('parallel')
%timeit -n 1 -r 1 Parallel(n_jobs=-1)(delayed(get_stability)(ref) for ref in refs)

parallel
540 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:
# accumulate stabilities
stabilities = Parallel(n_jobs=-1)(delayed(get_stability)(ref) for ref in tqdm(refs))

  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
# Collect all binodals and drop any with None (super crit)
binodals = [bino for s in stabilities if (bino := s['bino']) is not None]

In [23]:
# collect properties and concat
table = xr.concat([b.xge.table(keys=['dens_tot', 'pressure']) for b in binodals], dim='sample')
table.to_dataframe()

binodal     lnz_0  phase  temp      beta  \
binodal lnz_0     phase component                                             
0       -4.106914 0     0                0 -4.106914      0  0.95  1.052632   
                  1     0                0 -4.106914      1  0.95  1.052632   
        -3.594936 0     0                0 -3.594936      0  1.05  0.952381   
                  1     0                0 -3.594936      1  1.05  0.952381   
        -4.418965 0     0                0 -4.418965      0  0.90  1.111111   
                  1     0                0 -4.418965      1  0.90  1.111111   
        -3.834420 0     0                0 -3.834420      0  1.00  1.000000   
                  1     0                0 -3.834420      1  1.00  1.000000   
        -4.778798 0     0                0 -4.778798      0  0.85  1.176471   
                  1     0                0 -4.778798      1  0.85  1.176471   
        -3.197065 0     0                0 -3.197065      0  1.15  0.869565   
                  1     0                0 -3.197065      1  1.15  0.869565   
        -3.030589 0     0                0 -3.030589      0  1.20  0.833333   
                  1     0                0 -3.030589      1  1.20  0.833333   
        -5.683363 0     0                0 -5.683363      0  0.75  1.333333   
                  1     0                0 -5.683363      1  0.75  1.333333   
        -3.384181 0     0                0 -3.384181      0  1.10  0.909091   
                  1     0                0 -3.384181      1  1.10  0.909091   
        -5.195706 0     0                0 -5.195706      0  0.80  1.250000   
                  1     0                0 -5.195706      1  0.80  1.250000   
        -6.256642 0     0                0 -6.256642      0  0.70  1.428571   
                  1     0                0 -6.256642      1  0.70  1.428571   

                                   volume tail  rcut  nphase  dens_tot  \
binodal lnz_0     phase component                                        
0       -4.106914 0     0           512.0  LRC   3.0       2  0.021019   
                  1     0           512.0  LRC   3.0       2  0.727630   
        -3.594936 0     0           512.0  LRC   3.0       2  0.040668   
                  1     0           512.0  LRC   3.0       2  0.672076   
        -4.418965 0     0           512.0  LRC   3.0       2  0.014512   
                  1     0           512.0  LRC   3.0       2  0.752744   
        -3.834420 0     0           512.0  LRC   3.0       2  0.029566   
                  1     0           512.0  LRC   3.0       2  0.701151   
        -4.778798 0     0           512.0  LRC   3.0       2  0.009640   
                  1     0           512.0  LRC   3.0       2  0.776914   
        -3.197065 0     0           512.0  LRC   3.0       2  0.074110   
                  1     0           512.0  LRC   3.0       2  0.605182   
        -3.030589 0     0           512.0  LRC   3.0       2  0.100375   
                  1     0           512.0  LRC   3.0       2  0.563140   
        -5.683363 0     0           512.0  LRC   3.0       2  0.003630   
                  1     0           512.0  LRC   3.0       2  0.821441   
        -3.384181 0     0           512.0  LRC   3.0       2  0.055064   
                  1     0           512.0  LRC   3.0       2  0.640863   
        -5.195706 0     0           512.0  LRC   3.0       2  0.006103   
                  1     0           512.0  LRC   3.0       2  0.799605   
        -6.256642 0     0           512.0  LRC   3.0       2  0.001997   
                  1     0           512.0  LRC   3.0       2  0.843478   

                                   pressure        nvec     betapV      PE_n  
binodal lnz_0     phase component                                             
0       -4.106914 0     0          0.017534   10.761982   9.450089 -0.205928  
                  1     0          0.017534  372.546725   9.450089 -5.109548  
        -3.594936 0     0          0.034368  